<a href="https://colab.research.google.com/github/rameshveer/School_of_AI-EVA_5/blob/master/S11/EVA5_S11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# S11 Assignment:

## Write a code such that

### 1. Modified Resnet Architecture
  uses the ResNet Architecture for Cifar10:

### 2. Uses One Cycle Policy such that:
  Total Epochs = 24
  Max at Epoch = 5
  LRMIN = FIND
  LRMAX = FIND
  NO Annihilation
### 3. Uses this transform -
  >> RandomCrop 32, 32 (after padding of 4) 

  >> FlipLR 

  >> Followed by CutOut(8, 8)

  >> Batch size = 512
  
  >> Target Accuracy: 90%.

In [1]:
%matplotlib inline

Install albumentation code

In [2]:
!pip install albumentations==0.4.5

     |████████████████████████████████| 122kB 8.0MB/s 
     |████████████████████████████████| 634kB 16.5MB/s 
  Created wheel for albumentations: filename=albumentations-0.4.5-cp36-none-any.whl size=64378 sha256=940b2a9030309f76a4c93a4b2681d149dd534f1251d784417016237928ec1ff7
  Stored in directory: /root/.cache/pip/wheels/f0/a0/61/e50f93165a5ec7e7f5d65064e513239505bc4c06d2289557d3
  Created wheel for imgaug: filename=imgaug-0.2.6-cp36-none-any.whl size=654021 sha256=05b13b2abc9b179bce4baaa5b96fd74d13284a4c4425dd0ee94f10c31ed2e2c0
  Stored in directory: /root/.cache/pip/wheels/97/ec/48/0d25896c417b715af6236dbcef8f0bed136a1a5e52972fc6d0
Successfully built albumentations imgaug
  Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9
  Found existing installation: albumentations 0.1.12
    Uninstalling albumentations-0.1.12:
      Successfully uninstalled albumentations-0.1.12


### Import Created libraries (Specific to this excercise)

In [3]:
rm -rf convlib

In [4]:
!git clone https://github.com/rameshveer/convlib.git

Cloning into 'convlib'...
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 157 (delta 6), reused 0 (delta 0), pack-reused 138
Receiving objects: 100% (157/157), 96.60 KiB | 867.00 KiB/s, done.
Resolving deltas: 100% (66/66), done.


In [5]:
!pip install -e /content/convlib

Obtaining file:///content/convlib
  Running setup.py develop for convlib


### Importing Torch libraries


In [6]:
import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from torchsummary import summary

### Import created libraries 

In [7]:
import convlib.models.resnet as model
import convlib.utils.cuda as cu
import convlib.utils.dataloader as dataloader
import convlib.utils.trainer as trainer
import convlib.utils.evaluate as evaluate
from convlib.data.dataset import CIFAR10


In [8]:
#Setup params

class Args:
    # Training
    # ========
    random_seed = 1
    epochs = 24
    learning_rate = 0.01
    momentum = 0.9
    lr_step_size = 25
    lr_gamma = 0.1
    batch_size = 64
    num_workers=4
    sample_count = 25

    # Augmentation
    # ============
    horizontal_flip_prob = 0.2
    vertical_flip_prob = 0.0
    gaussian_blur_prob = 0.0
    rotate_degree = 20
    cutout = 0.3

    # Means & std:
    means = (0.5, 0.5, 0.5)
    stdevs = (0.5, 0.5, 0.5)


### Setup Device

In [9]:
cuda, device = cu.init_cuda(Args.random_seed)
cuda

CUDA Available? True
Device: cuda


True

Apply Albumentation params in Transforms

In [10]:
import albumentations as A

dataset = CIFAR10(
    train_batch_size=Args.batch_size,
    val_batch_size=Args.batch_size,
    cuda=cuda,
    num_workers=Args.num_workers,
    horizontal_flip_prob=Args.horizontal_flip_prob,
    vertical_flip_prob=Args.vertical_flip_prob,
    gaussian_blur_prob=Args.gaussian_blur_prob,
    rotate_degree=Args.rotate_degree,
    cutout=Args.cutout
)

Extracting /content/convlib/data/cifar10/cifar-10-python.tar.gz to /content/convlib/data/cifar10
Files already downloaded and verified
Files already downloaded and verified


In [11]:
# Create train data loader
train_loader = dataset.loader(train=True)

# Create val data loader
val_loader = dataset.loader(train=False)

In [12]:

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Summary of model

In [13]:
from torchsummary import summary

print(device)
model = model.ResNet18().to(device)
summary(model,input_size = (3,32,32))

cuda
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           1,728
       BatchNorm2d-2           [-1, 64, 32, 32]             128
            Conv2d-3           [-1, 64, 32, 32]          36,864
       BatchNorm2d-4           [-1, 64, 32, 32]             128
            Conv2d-5           [-1, 64, 32, 32]          36,864
       BatchNorm2d-6           [-1, 64, 32, 32]             128
        BasicBlock-7           [-1, 64, 32, 32]               0
            Conv2d-8           [-1, 64, 32, 32]          36,864
       BatchNorm2d-9           [-1, 64, 32, 32]             128
           Conv2d-10           [-1, 64, 32, 32]          36,864
      BatchNorm2d-11           [-1, 64, 32, 32]             128
       BasicBlock-12           [-1, 64, 32, 32]               0
           Conv2d-13          [-1, 128, 32, 32]          73,728
      BatchNorm2d-14          [-1,

Train the model

In [ ]:
#Loss function and Optimizer - Let's use a Classification Cross-Entropy loss and SGD with momentum.

criterion = nn.CrossEntropyLoss()

from torch.optim.lr_scheduler import StepLR
import torch.optim as optim

losses = []
accuracies = []
correct_samples = []
incorrect_samples = []

optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
scheduler = StepLR(optimizer, step_size=6, gamma=0.3)
last_epoch = False

for epoch in range(1, Args.epochs + 1):
    print(f'Epoch {epoch}:')
    if epoch == Args.epochs:
        last_epoch = True
    trainer.train(model, train_loader, device, optimizer, criterion)
    scheduler.step()
    trainer.val(
        model, val_loader, device, criterion, losses, accuracies,
        correct_samples, incorrect_samples, Args.sample_count, last_epoch=last_epoch
    )


  0%|          | 0/782 [00:00<?, ?it/s]

Epoch 1:


  0%|          | 0/782 [00:00<?, ?it/s]


Validation set: Average loss: 0.0140, Accuracy: 6800/10000 (68.00%)

Epoch 2:


  0%|          | 0/782 [00:00<?, ?it/s]

...

  0%|          | 0/782 [00:00<?, ?it/s]


Validation set: Average loss: 0.0099, Accuracy: 7841/10000 (78.41%)

Epoch 4:


  0%|          | 0/782 [00:00<?, ?it/s]


Validation set: Average loss: 0.0103, Accuracy: 7848/10000 (78.48%)

Epoch 5:


Loss=0.61 Batch_ID=329 Accuracy=82.87:  42%|████▏     | 329/782 [00:28<00:38, 11.67it/s]